# Analyisis of Mexican Judicial Reform Sentiment

I chose to explore the topic of judicial reform in Mexico because it is currently of significant importance. 
I was genuinely interested in examining whether the tone and wording of certain newspapers leaned more in favor of or against this issue. 

Initially, I utilized Google News for data collection; however, despite its usefulness, I encountered difficulties filtering results by date. I attempted several solutions from the documentation, but the parameters did not work as expected.
To analyze sentiment, I employed TextBlob, which revealed that the majority of articles had a neutral or positive tone. I speculated that this might be due to the articles being written after the reform was approved. 
To investigate further, I turned to NewsAPI, where I could easily filter articles by date. I set the dates prior to September 11, the day of the reform's approval, to conduct another sentiment analysis. To my surprise, I found that nearly all articles remained neutral or positive. Following this, I attempted to scrape opposition newspapers to balance my sample. However, I encountered issues with dynamic content, and due to time constraints, I could not complete the scraping process. I made an initial attempt using Selenium for this purpose.

Afterward, I proceeded with vectorization and trained a Multinomial Naive Bayes (MNB) model. The results indicate that while the model performed well in identifying neutral and positive sentiments, it struggled with negative sentiment classification, highlighting areas for improvement in future analyses.

In [122]:
#import libraries
from GoogleNews import GoogleNews
from newspaper import Article
import pandas as pd
import time
import random
from datetime import datetime, timedelta
from textblob import TextBlob
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

In [100]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

In [48]:
googlenews = GoogleNews()

#function to start the search for an specific query
def get_news(query, num_articles=100):
    googlenews.search(query)
    
    articles = []
    page = 1
    
    while len(articles) < num_articles:
        result = googlenews.result()
        for item in result:
            if len(articles) < num_articles:
                articles.append(item)
            else:
                break
                
        page += 1
        googlenews.get_page(page)
    
    return articles

#I realized that Google News appends identifiers to URLs, which trigger bot detection when scraping with newspaper
#this function removes these identifiers (after "&") 
def clean_url(url):
    cleaned_url = url.split('&')[0]
    return cleaned_url

#function to extract the full text of an article from its URL
def get_full_text(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        return article.text
    except Exception as e:
        print(f"Error fetching article: {e}")
        return "Text not available"

In [49]:
#we define our topic of interest, which is "reforma judicial"
query = "reforma judicial"
news = get_news(query, 300)

df = pd.DataFrame(news)
df = df[['date', 'media', 'title', 'desc', 'link']]

df['text'] = ""

for idx, row in df.iterrows():
    print(f"Fetching text for article {idx+1} of {len(df)}")
    cleaned_link = clean_url(row['link'])
    full_text = get_full_text(cleaned_link)
    df.at[idx, 'text'] = full_text
    time.sleep(random.uniform(2, 5))

#we save the data into an excel file for the analysis
df.to_excel('news_judicial_reform_all.xlsx', index=False)
print("News saved to 'news_judicial_reform_all.xlsx'")

Fetching text for article 1 of 300
Fetching text for article 2 of 300
Fetching text for article 3 of 300


<html lang="en">
<head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <meta name="description" content="px-captcha">
    <titl


Error fetching article: Article `download()` failed with Website protected with PerimeterX, url: None on URL https://www.ctinsider.com/news/world/article/el-senado-de-mexico-aprueba-una-controversial-19756945.php
Fetching text for article 4 of 300


<html lang="en">
<head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <meta name="description" content="px-captcha">
    <titl


Error fetching article: Article `download()` failed with Website protected with PerimeterX, url: None on URL https://www.manisteenews.com/news/world/article/el-senado-de-mexico-aprueba-una-controversial-19756945.php
Fetching text for article 5 of 300
Fetching text for article 6 of 300


...

<html lang="en">
<head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <meta name="description" content="px-captcha">
    <titl


Error fetching article: Article `download()` failed with Website protected with PerimeterX, url: None on URL https://www.ctpost.com/news/world/article/el-senado-de-mexico-aprueba-una-controversial-19756945.php
Fetching text for article 295 of 300
Fetching text for article 296 of 300
Fetching text for article 297 of 300


<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-


Error fetching article: Article `download()` failed with Website protected with Cloudflare, url: None on URL https://headtopics.com/us/manifestantes-irrumpen-en-el-senado-de-mexico-durante-58814315
Fetching text for article 298 of 300
Fetching text for article 299 of 300
Fetching text for article 300 of 300


Error fetching article: Article `download()` failed with Status code 403 for url None on URL https://www.wsj.com/world/americas/mexican-legislature-passes-judicial-overhaul-5399e8ff
News saved to 'news_judicial_reform_all.xlsx'


In [50]:
df = pd.read_excel('news_judicial_reform_all.xlsx')
#filter out articles where the text could not be downloaded
df_filtered = df[df['text'] != "Text not available"]
df_filtered.head(3)

,date,media,title,desc,link,text
0,3 hours ago,Bloomberg,Legislaturas estatales de México respaldan ref...,Más de la mitad de las legislaturas estatales ...,https://www.bloomberg.com/news/articles/2024-0...,Más de la mitad de las legislaturas estatales ...
1,4 hours ago,The New York Times,México está dividido frente al nuevo plan para...,Mientras una amplia propuesta para elegir casi...,https://www.nytimes.com/es/2024/09/12/espanol/...,[Estamos en WhatsApp. Empieza a seguirnos ahor...
4,1 day ago,Bloomberg,Inversores advierten sobre retroceso a 1970 po...,"En los últimos seis años, los inversionistas s...",https://www.bloomberg.com/news/articles/2024-0...,"En los últimos seis años, los inversionistas s..."


In [54]:
#calculate the subjectivity, polarity and get the analysis using library TextBlob
def get_subjectivity(text):
    if isinstance(text, str): 
        return TextBlob(text).sentiment.subjectivity
    return None

def get_polarity(text):
    if isinstance(text, str):
        return TextBlob(text).sentiment.polarity
    return None

def get_analysis(score):
    if score is None:
        return 'No analysis' 
    if score < 0:
        return 'Negative'
    elif score == 0:
        return 'Neutral'
    else:
        return 'Positive'

In [55]:
df_filtered['TextBlob_Subjectivity'] = df_filtered['text'].apply(get_subjectivity)
df_filtered['TextBlob_Polarity'] = df_filtered['text'].apply(get_polarity)
df_filtered['TextBlob_Analysis'] = df_filtered['TextBlob_Polarity'].apply(get_analysis)

print(df_filtered[['title', 'TextBlob_Subjectivity', 'TextBlob_Polarity', 'TextBlob_Analysis']].head())

/var/folders/17/fn1y9q8s0svbp4zb8gr_707h0000gp/T/ipykernel_1403/108850679.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['TextBlob_Subjectivity'] = df_filtered['text'].apply(get_subjectivity)


                                                title  TextBlob_Subjectivity  \
0   Legislaturas estatales de México respaldan ref...               0.900000   
1   México está dividido frente al nuevo plan para...               0.677273   
4   Inversores advierten sobre retroceso a 1970 po...               0.000000   
6                  Judicial Reform Approved in Mexico               0.000000   
10  Legislaturas estatales de México respaldan ref...               0.900000   

    TextBlob_Polarity TextBlob_Analysis  
0            0.600000          Positive  
1            0.368182          Positive  
4            0.000000           Neutral  
6            0.000000           Neutral  
10           0.600000          Positive  


/var/folders/17/fn1y9q8s0svbp4zb8gr_707h0000gp/T/ipykernel_1403/108850679.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['TextBlob_Polarity'] = df_filtered['text'].apply(get_polarity)
/var/folders/17/fn1y9q8s0svbp4zb8gr_707h0000gp/T/ipykernel_1403/108850679.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['TextBlob_Analysis'] = df_filtered['TextBlob_Polarity'].apply(get_analysis)


In [57]:
df_filtered.to_excel('news_judicial_reform_sentiment_analysis.xlsx', index=False)

In [72]:
#I felt that the articles scrpaed previously were insufficient, as I had difficulty obtaining negative sentiments
#so I readed that NewsAPI is a good source for collecting news articles through an API, which piqued my interest in exploring it further

#I created an user account on NewsAPI to obtain my api key
api_key = '06d4f092dd26487ab3b6f2c44531b60c'

# Function to get news articles
def get_news(query, from_date, to_date):
    url = f'https://newsapi.org/v2/everything?q={query}&from={from_date}&to={to_date}&language=es&apiKey={api_key}'
    response = requests.get(url)
    data = response.json()
    
    if response.status_code == 200 and data['status'] == 'ok':
        return data['articles']
    else:
        print("Error fetching news:", data.get('message', ''))
        return []

# Function to scrape full text from article URL
def scrape_full_text(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        paragraphs = soup.find_all('p')
        full_text = ' '.join([para.get_text() for para in paragraphs])
        return full_text
    except Exception as e:
        print(f"Error fetching article text: {e}")
        return "Text not available"

In [81]:
query = "reforma judicial"
#I selected these dates before the reform's approval on September 11, 2024 to check if the tone of the artciles was different
from_date = "2024-09-01"
to_date = "2024-09-10"

articles = get_news(query, from_date, to_date)
#iterate over each article to extract relevant information
news_data = []
for article in articles:
    title = article['title']
    link = article['url']
    published_at = article['publishedAt']
    full_text = scrape_full_text(link) 
    
    news_data.append({
        'title': title,
        'link': link,
        'date': published_at,
        'text': full_text
    })

df_before = pd.DataFrame(news_data)
df.to_excel('judicial_reform_articles_before.xlsx', index=False)
print("Articles saved to 'judicial_reform_articles_before.xlsx'")

Articles saved to 'judicial_reform_articles_before.xlsx'


In [85]:
df_before = pd.read_excel('judicial_reform_articles_before.xlsx')
df_before.head(3)

,title,link,publishedAt,full_text
0,Cámara de Diputados aprueba en lo general la r...,https://cnnespanol.cnn.com/2024/09/04/camara-d...,2024-09-04T11:04:48Z,"\n(CNN Español) – Con 359 votos a favor, 135 e..."
1,"Reforma judicial de AMLO en México, en vivo: ú...",https://cnnespanol.cnn.com/2024/09/04/reforma-...,2024-09-04T12:27:55Z,\n© 2024 Cable News Network.\nA Warner Bros. D...
2,3 claves para entender qué cambia con la polém...,https://www.bbc.com/mundo/articles/cx28kzy4g8ko,2024-09-05T13:49:14Z,"Fuente de la imagen, Getty Images La controver..."


In [93]:
df_before = df_before[df_before['full_text'].notna() & (df_before['full_text'].str.strip() != "")]
df_before['publishedAt'] = pd.to_datetime(df_before['publishedAt']).dt.strftime('%Y-%m-%d %H:%M:%S')
df_before.head(20)

,title,link,publishedAt,full_text,TextBlob_Subjectivity,TextBlob_Polarity,TextBlob_Analysis
0,Cámara de Diputados aprueba en lo general la r...,https://cnnespanol.cnn.com/2024/09/04/camara-d...,2024-09-04 11:04:48,"\n(CNN Español) – Con 359 votos a favor, 135 e...",0.526667,0.303333,Positive
1,"Reforma judicial de AMLO en México, en vivo: ú...",https://cnnespanol.cnn.com/2024/09/04/reforma-...,2024-09-04 12:27:55,\n© 2024 Cable News Network.\nA Warner Bros. D...,0.476190,0.212585,Positive
2,3 claves para entender qué cambia con la polém...,https://www.bbc.com/mundo/articles/cx28kzy4g8ko,2024-09-05 13:49:14,"Fuente de la imagen, Getty Images La controver...",0.472727,0.218182,Positive
3,Una jueza ordena suspender la discusión de la ...,https://cnnespanol.cnn.com/2024/09/01/jueza-su...,2024-09-02 02:05:01,\n(CNN Español) –– Una jueza federal de México...,0.475000,0.162500,Positive
4,¿En qué consiste la reforma judicial que impul...,https://cnnespanol.cnn.com/2024/09/03/reforma-...,2024-09-03 19:40:13,\n(CNN Español) –El pleno de la Cámara de Dipu...,0.590000,0.280000,Positive
5,López Obrador dice que la orden que suspende l...,https://cnnespanol.cnn.com/2024/09/02/lopez-ob...,2024-09-02 20:33:36,"\n(CNN Español) –– El presidente de México, A...",0.900000,0.400000,Positive
6,ANÁLISIS | ¿Por qué genera tanta polémica la r...,https://cnnespanol.cnn.com/2024/09/04/analisis...,2024-09-04 18:55:37,\n(CNN Español) – Desde que fue planteada por ...,0.592308,0.296154,Positive
7,Diputados de Morena anuncian que discusión de ...,https://cnnespanol.cnn.com/2024/09/03/diputado...,2024-09-03 22:55:38,\n© 2024 Cable News Network.\nA Warner Bros. D...,0.416667,0.108333,Positive
8,Estos son los pasos necesarios para aprobar la...,https://cnnespanol.cnn.com/2024/09/04/reforma-...,2024-09-04 14:17:16,\n© 2024 Cable News Network.\nA Warner Bros. D...,0.456667,0.108333,Positive
9,El juez Llarena da un nuevo portazo a Puigdemo...,https://www.elmundo.es/espana/2024/09/10/66e01...,2024-09-10 09:32:55,Portada \nSUSCRÍBETE\n20%\n DTO.\n SUSCRÍBET...,0.384848,0.146970,Positive


In [94]:
#I analized the articles using TextBlob again
#I found thtat the tone of nearly all article before the approval was neutral or postive (again)
df_before['TextBlob_Subjectivity'] = df_before['full_text'].apply(get_subjectivity)
df_before['TextBlob_Polarity'] = df_before['full_text'].apply(get_polarity)
df_before['TextBlob_Analysis'] = df_before['TextBlob_Polarity'].apply(get_analysis)

print(df_before[['title', 'TextBlob_Subjectivity', 'TextBlob_Polarity', 'TextBlob_Analysis']].head())

                                               title  TextBlob_Subjectivity  \
0  Cámara de Diputados aprueba en lo general la r...               0.526667   
1  Reforma judicial de AMLO en México, en vivo: ú...               0.476190   
2  3 claves para entender qué cambia con la polém...               0.472727   
3  Una jueza ordena suspender la discusión de la ...               0.475000   
4  ¿En qué consiste la reforma judicial que impul...               0.590000   

   TextBlob_Polarity TextBlob_Analysis  
0           0.303333          Positive  
1           0.212585          Positive  
2           0.218182          Positive  
3           0.162500          Positive  
4           0.280000          Positive  


In [95]:
df_before.to_excel('news_judicial_reform_sentiment_analysis_before.xlsx', index=False)

In [96]:
df_before.head(3)

,title,link,publishedAt,full_text,TextBlob_Subjectivity,TextBlob_Polarity,TextBlob_Analysis
0,Cámara de Diputados aprueba en lo general la r...,https://cnnespanol.cnn.com/2024/09/04/camara-d...,2024-09-04 11:04:48,"\n(CNN Español) – Con 359 votos a favor, 135 e...",0.526667,0.303333,Positive
1,"Reforma judicial de AMLO en México, en vivo: ú...",https://cnnespanol.cnn.com/2024/09/04/reforma-...,2024-09-04 12:27:55,\n© 2024 Cable News Network.\nA Warner Bros. D...,0.476190,0.212585,Positive
2,3 claves para entender qué cambia con la polém...,https://www.bbc.com/mundo/articles/cx28kzy4g8ko,2024-09-05 13:49:14,"Fuente de la imagen, Getty Images La controver...",0.472727,0.218182,Positive


In [106]:
# Combine df_before and df_filtered for analysis
df_combined = pd.concat([
    df_before[['full_text', 'TextBlob_Analysis']].rename(columns={'full_text': 'text'}),
    df_filtered[['text', 'TextBlob_Analysis']]
], ignore_index=True)

df_combined = df_combined.rename(columns={'TextBlob_Analysis': 'sentiment'})

# Drop NaN values in the 'text' column
df_combined = df_combined.dropna(subset=['text', 'sentiment'])

# Split the data into features and labels
X = df_combined['text']
y = df_combined['sentiment']

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorize the text
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Train a Naives Bayes Model
model = MultinomialNB()
model.fit(X_train_vectorized, y_train)

# Make predictions
y_pred = model.predict(X_test_vectorized)

# Evaluate the model
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    Negative       0.00      0.00      0.00         1
     Neutral       0.75      0.91      0.82        23
    Positive       0.86      0.72      0.78        25

    accuracy                           0.80        49
   macro avg       0.54      0.54      0.54        49
weighted avg       0.79      0.80      0.79        49



/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
